# Regresión espacial 

Consideraremos dos procesos relacionados pero muy diferentes que dan lugar a efectos espaciales: *Heterogeneidad Espacial (SH)* y *Dependencia Espacial (SD)*.

La heterogeneidad espacial se refiere a la variación en las propiedades de los datos en diferentes áreas del espacio, lo que implica que las relaciones entre las variables no necesariamente sean homogéneas en toda la región de estudio (Rey et al. 2023). La dependencia espacial, por otro lado, se refiere a la tendencia de valores similares a agruparse espacialmente (Anselin 1988). Esto implica que el valor de una variable en un punto del espacio está correlacionado con los valores de la misma variable en puntos cercanos, un fenómeno también conocido como autocorrelación. 

Los modelos clásicos suelen asumir que las observaciones son independientes entre sí (sin dependencia espacial) y que la relación entre las variables es constante en toda la región de estudio (sin heterogeneidad espacial) (Lesage 2011, Banerjee 2008).

# Dependencia espacial


### 1. Modelos de regresión para dependencia espacial tipo SAR (Modelos autorregresivos espaciales)

Para incorporar la dependencia espacial en los modelos, se puede utilizar los llamados Modelos Autoregresivos Simultáneos (Whittle 1954), los cuales permiten modelar la dependencia espacial a través de la inclusión de una matriz de vecindad. Esta matriz define la estructura de los vecinos de cada unidad de mapeo, ya sea en función de la distancia entre unidades o a partir de relaciones de contigüidad (Anselin 1996, Elhorst 2014).

Los modelos de autoregresión espacial, en lugar de asumir que las observaciones son independientes entre sí, reconocen que las unidades cercanas en el espacio tienden a presentar valores similares (Anselin 1988`).

$$Y = \rho W Y + X \beta + \epsilon$$ 


In [1]:
import geopandas as gpd
import numpy as np
import libpysal
from libpysal.weights import Queen
from sklearn.preprocessing import StandardScaler
import spreg

# Cargar datos
gdf = gpd.read_file(r"G:\My Drive\Investigacion2025\Posgrado_Statistics\GeoAnalysis\data\Balso_Departamento.gpkg")
gdf.info()

# Renombramos variables
gdf = gdf.rename(columns={'NOMBRE_DPT': 'departamento', 
                    'AREA_OFICI': 'area',
                    'NUMPOINTS': 'conteo'})
gdf.head(3)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   ID_ESPACIA                               34 non-null     object  
 1   AREA_OFICI                               34 non-null     float64 
 2   NOMBRE_DPT                               34 non-null     object  
 3   NUMPOINTS                                34 non-null     float64 
 4   elev_mean                                34 non-null     float64 
 5   Temperatura_media_anual_mean             34 non-null     float64 
 6   Precipitacion_anual_mean                 34 non-null     float64 
 7   Rango_medio_diurno_mean                  34 non-null     float64 
 8   Precipitacion_mes_mas_lluvioso_mean      34 non-null     float64 
 9   Precipitacion_mes_mas_seco_mean          34 non-null     float64 
 10  Isotermalidad_mean              

,ID_ESPACIA,area,departamento,conteo,elev_mean,Temperatura_media_anual_mean,Precipitacion_anual_mean,Rango_medio_diurno_mean,Precipitacion_mes_mas_lluvioso_mean,Precipitacion_mes_mas_seco_mean,Isotermalidad_mean,Estacionalidad_de_la_temperatura_mean,Rango_anual_de_temperatura_mean,Estacionalidad_de_la_precipitacion_mean,geometry
0,11,1605.0,BOGOTÁ,0.0,3223.298701,9.803929,1266.961039,9.179502,168.558442,39.285714,80.775546,35.473980,11.352364,38.033142,"MULTIPOLYGON (((4881435.89 2091783.395, 488145..."
1,13,26383.0,BOLÍVAR,32.0,263.826877,26.778483,2270.948617,9.940637,355.376285,29.230040,83.090072,42.905979,12.014608,59.258936,"MULTIPOLYGON (((4753419.752 2752315.572, 47534..."
2,15,23189.0,BOYACÁ,6.0,2262.947658,15.462887,1776.909091,9.494651,279.260790,36.449036,80.130047,43.557213,11.859600,53.210623,"MULTIPOLYGON (((5092792.513 2336225.178, 50929..."


In [15]:
gdf.columns

Index(['ID_ESPACIA', 'area', 'departamento', 'conteo', 'elev_mean',
       'Temperatura_media_anual_mean', 'Precipitacion_anual_mean',
       'Rango_medio_diurno_mean', 'Precipitacion_mes_mas_lluvioso_mean',
       'Precipitacion_mes_mas_seco_mean', 'Isotermalidad_mean',
       'Estacionalidad_de_la_temperatura_mean',
       'Rango_anual_de_temperatura_mean',
       'Estacionalidad_de_la_precipitacion_mean', 'geometry'],
      dtype='object')

In [16]:
# Renombrar variables en el gdf original a nombres cortos
gdf = gdf.rename(columns={
    'elev_mean': 'elev',
    'Temperatura_media_anual_mean': 'temp',
    'Precipitacion_anual_mean': 'precip',
    'Rango_medio_diurno_mean': 'rmd',
    'Precipitacion_mes_mas_lluvioso_mean': 'p_lluv',
    'Precipitacion_mes_mas_seco_mean': 'p_seco',
    'Isotermalidad_mean': 'isoterm',
    'Estacionalidad_de_la_temperatura_mean': 'est_temp',
    'Rango_anual_de_temperatura_mean': 'rango_temp',
    'Estacionalidad_de_la_precipitacion_mean': 'est_precip'
})

En un modelo OLS tradicional, la interpretación de un coeficiente es sencilla: si X aumenta en una unidad, Y cambia en β unidades, manteniendo todo lo demás constante.

### Modelo SLX (Spatial Lag of X)
Incluye rezagos espaciales de las variables independientes (WX) como predictores adicionales en un modelo OLS. 
Es útil cuando quieres ver cómo las características de áreas vecinas influyen sobre una región (por ejemplo, cómo el clima en municipios cercanos afecta el valor local de y).

In [19]:

# Aplica la transformación log + 1 a la varaible de interes
import pandas as pd

gdf['y'] = np.log(gdf['conteo'] + 1)

# Definir las variables dependiente e independientes
y = gdf['y'].values.reshape((-1, 1))
# Ahora define la lista de variables independientes con los nombres cortos
independent_vars = ['elev', 'temp', 'precip', 'rmd', 'p_lluv',
    'p_seco', 'isoterm', 'est_temp', 'rango_temp', 'est_precip']
X = gdf[independent_vars].values

# Escalar las variables independientes
st = StandardScaler()
X_scaled = st.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=independent_vars)

# Crear la matriz de pesos espaciales por contigüidad tipo Queen
w = Queen.from_dataframe(gdf)

# Calcular la autocorrelación exógena de las variables independientes escaladas
wx_dict = {}
for var in independent_vars:
    wx_dict[f'w_{var}'] = libpysal.weights.spatial_lag.lag_spatial(w, X_scaled_df[var])

wx_df = pd.DataFrame(wx_dict)

# Combinar las variables independientes originales escaladas con sus versiones espacialmente rezagadas
slx_exog = pd.concat([X_scaled_df, wx_df], axis=1)

# Ajustar el modelo OLS con las variables exógenas espacialmente rezagadas (SLX)
ols_model_slx = spreg.OLS(gdf['y'].values.reshape((-1, 1)),
                          slx_exog.values,
                          name_y='log_conteo',
                          name_x=slx_exog.columns.tolist(),
                          name_w='queen_contiguity')

# Imprimir el resumen del modelo OLS con SLX
print(ols_model_slx.summary)

with open("resumen_modelo.txt", "w") as f:
    f.write(str(ols_model_slx.summary))

C:\Users\Valentina Cardona\AppData\Local\Temp\ipykernel_22616\1722801646.py:19: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :  log_conteo                Number of Observations:          34
Mean dependent var  :      2.7670                Number of Variables   :          21
S.D. dependent var  :      1.9525                Degrees of Freedom    :          13
R-squared           :      0.8612
Adjusted R-squared  :      0.6476
Sum squared residual:     17.4641                F-statistic           :      4.0324
Sigma-square        :       1.343                Prob(F-statistic)     :    0.006512
S.E. of regression  :       1.159                Log likelihood        :     -36.918
Sigma-square ML     :       0.514                Akaike info criterion :     115.837
S.E of regression ML:      0.7167                Schwarz criterion     :     147.890

------------------------------------------------------------

c:\Users\Valentina Cardona\anaconda3\envs\GeoAnalysis\Lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
 There is 1 island with id: 25.
  W.__init__(self, neighbors, ids=ids, **kw)


### Modelo de error espacial — GM_Error_Het:

Modela la dependencia espacial en los errores del modelo, no en las variables ni en la variable dependiente.

Principal uso: 
- Hay evidencia de autocorrelación en los residuos de un modelo OLS.
- El fenómeno espacial no está directamente relacionado con las variables independientes, sino con factores no observados.

In [ ]:
# Ajustar el modelo de error espacial (GM_Error_Het)
spatial_error_model = spreg.GM_Error_Het(y, X_scaled, w=w,
                                      name_y='log_conteo',
                                      name_x=independent_vars,
                                      name_w='queen_contiguity')

# Imprimir el resumen del modelo de error espacial
print(spatial_error_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: GM SPATIALLY WEIGHTED LEAST SQUARES (HET)
------------------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :log_density_lands_rec                Number of Observations:          34
Mean dependent var  :      0.0052                Number of Variables   :          11
S.D. dependent var  :      0.0174                Degrees of Freedom    :          23
Pseudo R-squared    :      0.3526
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.00514         0.00170         3.01851         0.00254
                elev         0.07442       

### Modelo de retardo espacial (SAR) — GM_Lag

Incluye rezagos espaciales de la variable dependiente (Wy) como regresor. Es un modelo de interdependencia directa entre las unidades. Implica que el valor de y en una región depende de los valores de y en regiones vecinas.

Útil cuando: Los efectos espaciales son endógenos (es decir, se transmiten directamente entre unidades).

In [20]:
# Ajustar el modelo de retardo espacial (GM_Lag) =SAR
spatial_lag_model = spreg.GM_Lag(y, X_scaled, w=w,
                                name_y='log_conteo',
                                name_x=independent_vars,
                                name_w='queen_contiguity')

# Imprimir el resumen del modelo de retardo espacial
print(spatial_lag_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :  log_conteo                Number of Observations:          34
Mean dependent var  :      2.7670                Number of Variables   :          12
S.D. dependent var  :      1.9525                Degrees of Freedom    :          22
Pseudo R-squared    :      0.4734
Spatial Pseudo R-squared:  0.4771

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         2.48554         0.87660         2.83545         0.00458
                elev        13.30003         5.07197         2.62226         0.00873
                temp        13.51779  

### 2. Modelos de regresión para dependencia espacial tipo CAR (Modelos autorregresivos condicionales)

Rho típicamente varía entre 0 y 1 e indica la fuerza de la autocorrelación espacial. Un valor de 0 significa que no hay autocorrelación espacial, mientras que un valor más cercano a 1 indica una fuerte autocorrelación espacial.

A diferencia de los modelos CAR “propios” (donde rho < 1), la distribución conjunta de un modelo ICAR es “impropia”, lo que significa que no se integra a 1. Los modelos ICAR son particularmente útiles para modelar el suavizado espacial. Imponen una estructura donde las unidades vecinas tienen distribuciones posteriores que tienden a estar cerca unas de otras. Esto es útil cuando se espera que las áreas geográficamente cercanas tengan tasas o niveles de la variable de respuesta similares debido a factores espaciales no medidos. Los efectos espaciales en un modelo ICAR implican una restricción de suma a cero. Esto significa que la suma de los efectos espaciales a través de todas las unidades espaciales es cero. Al imponer la restricción de suma a cero, se fija un punto de referencia y se asegura que haya una única solución para los efectos espaciales que mejor se ajuste a los datos. 



Este modelo de regresión tiene desarrollo en Código de R. 

# Heterogeneidad espacial

 Modelos que permiten la variación espacial de los coeficientes de las variables predictoras, permitiendo capturar la heterogeneidad inherente al terreno (Rey et al. 2023). Entre estos enfoques se encuentran los modelos multiniveles (Wong 1985) (también conocidos como jerárquicos, y en algunos casos modelos mixtos o regímenes espaciales) y los modelos de regresión espacial ponderada (GWR, por sus siglas en inglés) (Brunsdon 1996). 
 
 Los modelos multiniveles permiten modelar la variabilidad espacial mediante la introducción de estructuras jerárquicas. Resulta particularmente útil cuando los datos presentan una organización natural en grupos o niveles (por ejemplo, cuencas y subcuencas). En contraste, los modelos GWR son menos complejos computacionalmente, ya que no requieren definir niveles jerárquicos. Los modelos GWR permiten que los coeficientes de las variables predictoras varíen para cada observación.

Los efectos fijos espaciales introducen econometricamente la noción de heterogeneidad espacial. Lo hacen en la forma más simple posible: al permitir que el término constante varíe geográficamente. Los demás elementos de la regresión permanecen sin cambios y, por lo tanto, se aplican uniformemente en todo el espacio. La idea de los regímenes espaciales (RE) es generalizar el enfoque de los efectos fijos espaciales para permitir que no solo el término constante varíe, sino también cualquier otra variable explicativa. Esto implica que la ecuación que estaremos estimando es:
 $$\log{P_i} = \alpha_r + \sum_k \mathbf{X}_{ki}\beta_{k-r} + \epsilon_i$$

donde no solo permitimos que el término constante varíe por región ($\alpha_r$
), sino también cada otro parámetro ($\beta_{k-r}$).

Código en R. Se realizo para el dataset de Municipios con efecto aleatorios por departamento. Nota: Evalue los efectos aleatorio a nivel de municipios y se generaba una matriz singular. 

# Modelos con dependencia y hetereogeneidad espacial

Una característica fundamental de los datos espaciales es la presencia simultánea de dependencia espacial y heterogeneidad espacial. La dependencia espacial se refiere al hecho de que las observaciones cercanas tienden a parecerse más entre sí que las distantes, debido a procesos subyacentes que operan en el espacio geográfico. Por su parte, la heterogeneidad espacial se manifiesta en diferencias sistemáticas entre regiones o ubicaciones, ya sea por condiciones ambientales, sociales o geológicas que varían en el territorio. 

* Estrategias para modelar conjuntamente dependencia y heterogenidad espacial:
1. Los modelos autoregresivos condicionales (CAR), implementados en la librería INLA, permiten incorporar simultáneamente estos dos aspectos en un marco bayesiano. La dependencia espacial se modela mediante una matriz de vecindad, y la heterogeneidad se introduce agregando un efecto aleatorio no estructurado (iid), como en el modelo BYM (Besag-York-Mollié). 

2. En la familia de modelos simultáneos autoregresivos (SAR) —implementados en R mediante la librería spatialreg— la dependencia espacial puede incorporarse a través de la variable dependiente (modelo SARlag), de los errores (SARerror), o de las covariables (modelo SLX). En estos modelos, la heterogeneidad espacial se representa explícitamente mediante interacciones entre las covariables continuas y un factor espacial categórico, como subcuencas o regiones, permitiendo que los efectos de las variables explicativas cambien según la unidad espacial. Este tipo de interacción permite capturar variaciones estructurales (regímenes espaciales) sin necesidad de introducir efectos aleatorios, y es una forma eficaz de modelar heterogeneidad espacial en un contexto frecuentista.